In [3]:
import folium
import pandas as pd

## 🗺️ Quick Visualization with Folium

To get an intuitive sense of the vessel trajectories, we plot the positions of a single ship (selected by its `MMSI`) on an interactive map using **Folium**.

This allows us to quickly verify that:
- The AIS points are correctly located within the Kattegat inspection area.  
- The temporal resampling (e.g., every 5 minutes) still preserves the vessel’s overall path.  
- The filtering and cleaning steps did not introduce spatial artifacts.

Each marker on the map corresponds to one AIS position for the selected vessel at a given timestamp.


In [ ]:
df = pd.read_csv("df_28_katt.csv")
df =pd.read_parquet("df_28_katt.parquet")

In [5]:
df_nave1 = df[df["MMSI"] == 220349000] #select mmsi

# Ensure 'Timestamp' is datetime and sort chronologically
df_nave1['Timestamp'] = pd.to_datetime(df_nave1['Timestamp'], errors='coerce')
df_nave1 = df_nave1.sort_values('Timestamp')

# Resample every  tot minutes
df_nave1_res = (
    df_nave1
    .set_index('Timestamp')
    .resample('125min')
    .first()              # take first observation in each 10-min bin
    .dropna(how='all')
    .reset_index()
)

df_nave1_res.shape


C:\Users\elefa\AppData\Local\Temp\ipykernel_3116\756694452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nave1['Timestamp'] = pd.to_datetime(df_nave1['Timestamp'], errors='coerce')


(12, 20)

In [6]:
# Create map centered on Kattegat/Læsø
map_center = [57.35, 11.0]  # [lat, lon]
m = folium.Map(
    location=map_center,
    zoom_start=9,
    tiles='OpenStreetMap'
)

coordinates = list(zip(df_nave1_res['Longitude'], df_nave1_res['Latitude']))

# Add vertex markers as triangles
for i, (lon, lat) in enumerate(coordinates[:-1]):  # Skip last (duplicate)
    folium.Marker(
        location=[lat, lon],
        popup=f'Vertex {i+1}<br>Lat: {lat:.4f}<br>Lon: {lon:.4f}',
        tooltip=f'V{i+1}',
        icon=folium.Icon(
            icon='f2ec',
            prefix='fa',  # Font Awesome icons
            color='red',
            icon_color='white'
        )
    ).add_to(m)


In [7]:
m